In [2]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_X


### 문제 1-1 : 기본 Chain 만들기 - AI 요리사

In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 1. 컴포넌트 정의
prompt = PromptTemplate.from_template("you are a professional chef. Answer the question. <Question>: {input}으로 만들 수 있는 요리를 추천해주고 간단한 레시피로를 알려주세요.")

# llm = ChatOpenAI(model="gpt-3.5-turbo")
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

output_parser = StrOutputParser()

# 2. chain 생성 (LCEL)
chain = prompt | llm | output_parser
print(type(chain))

# 3. chain의 invoke 호출
result = chain.invoke({"input": "토마토, 양파, 치즈"})
print(type(result))
print(result)

<class 'langchain_core.runnables.base.RunnableSequence'>
<class 'str'>
물론입니다. 토마토, 양파, 치즈만 있어도 정말 맛있는 **토마토 치즈 그라탕**을 간단하게 만들 수 있어요. 오븐 없이도 프라이팬으로 가능합니다.

---

### 🍅 **토마토 치즈 그라탕 (프라이팬 버전)**

#### ✅ 재료 (1~2인분)
- 토마토 2개  
- 양파 1/2개  
- 모짜렐라 치즈 (또는 피자 치즈) 적당량  
- 소금, 후추 약간  
- 올리브오일 (또는 식용유) 약간  

---

#### 🔪 간단 레시피
1. **토마토는 0.5cm 두께로 썰고, 양파는 얇게 채 썰어주세요.**
2. **프라이팬에 약간의 기름을 두르고, 양파를 먼저 볶아줍니다.** (약 2분간 투명해질 때까지)
3. **토마토를 올리고, 소금+후추로 간 합니다.** (약 2~3분간 중불에서 토마토가 살짝 말랑해지도록)
4. **위에 치즈를 듬뿍 올리고, 뚜껑을 덮어줍니다.** (약불에서 2~3분간 치즈가 완전히 녛을 때까지)

---

#### ✅ 팁
- 치즈가 눌어붙지 않도록 **불은 꼭 약불로** 조절하세요.
- 식빵이 있다면 옆에 구워서 함께 먹으면 **간단한 브런치**로도 딱입니다.

---

이 요리는 **준비 5분, 조리 10분**이면 끝나고, **새콤달콤한 토마토와 고소한 치즈**의 조합이 정말 맛있어요.


### 문제 1-2 : 2단계 Multi Chain 만들기 - 여행지 정보 시스템

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

from pprint import pprint

# Step 1: 사용자가 입력한 장르에 따라 영화 추천
prompt1 = ChatPromptTemplate.from_template("{input} 해당 지역의 대표적인 관광 명소 1가지를 추천해주세요.")

# Step 2: 추천된 영화의 줄거리를 요약
prompt2 = ChatPromptTemplate.from_template("{place} 추천한 명소를 먼저 알려주시고, 줄을 바꾸어서 추천받은 명소에 대한 상세 정보(역사, 특징, 방문 팁 등)를 제공해주세요.")

# OpenAI 모델 사용
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

# 체인 1: 명소 추천 (입력: 장르 → 출력: 명소 이름)
chain1 = prompt1 | llm | StrOutputParser()

# 체인 2: 줄거리 요약 (입력: 영화 제목 → 출력: 줄거리)
chain2 = (
    {"place": chain1}  # chain1의 출력을 place 변수로 전달
    | prompt2
    | llm
    | StrOutputParser()
)

# 실행: "Drama" 장르의 영화 추천 및 줄거리 요약
response = chain2.invoke({"input": "도쿄"})
print("\n🔹 명소에 대한 상세 정보:")
pprint(response)


🔹 명소에 대한 상세 정보:
('📍 추천 명소: 시부야 스카이(Shibuya Sky)  \n'
 '도쿄 시부야역 바로 옆 47층, 202 m 위에서 360°로 펼쳐지는 도쿄의 파노라마를 만날 수 있는 루프톱 전망대입니다. 발밑 투명 '
 '유리창 ‘스카이라인뷰’, 실외 헬리포트급 루프톱, 그리고 석양 뒤에 번쩍이는 스크램블 교차로의 네온은 도쿄 여행 필수 컷을 보장합니다.\n'
 '\n'
 '––––––––––––––––––––––––––––––\n'
 '시부야 스카이(Shibuya Sky) 상세 가이드\n'
 '––––––––––––––––––––––––––––––\n'
 '\n'
 '1. 역사  \n'
 '• 2019년 11월 개장. 시부야 가보리(渋谷川)를 따라 조성된 대형 복합빌딩 ‘시부야 스クランブ 스퀘어’ 최상층(45~47F)에 '
 '위치.  \n'
 '• 과거 ‘시부야 마크시티’와 ‘Bunkamura’가 있던 자리를 재개발하며 탄생한 랜드마크.\n'
 '\n'
 '2. 특징  \n'
 '• 360° 옥상 루프톱(屋上): 도쿄 타워·도쿄 스카이트리·후지산까지 한눈에.  \n'
 '• 스카이라인뷰(Skyline View): 바닥에 설치된 2 m 길이 투명 유리 바닥 구간.  \n'
 '• 실외 헬리쿠션(Heliport)급 데크: 바람을 맞으며 무장애 사진 촬영 가능.  \n'
 '• 인터랙티브 디지털 콘텐츠: AR千里眼(천리안) 스코프로 실시간 건물 정보·거리 표시.  \n'
 '• 음악·조명과 연동되는 ‘시부야 스카이 게이트’: 입장부터 몰입형 미디어 아트.  \n'
 '• 야경 타임(일몰~23:00): 매분 변하는 색온도·조도로 ‘도쿄 밤’ 하이라이트.\n'
 '\n'
 '3. 입장료 & 예약 팁  \n'
 '• 당일 현장표보다 공식 웹 예매가 200~400엔 저렴.  \n'
 '• 예약제 30분 단위 타임슬롯→지각 30분 초과 시 연기/취소 불가.  \n'
 '• 일출 체험 ‘선라이즈 티켓’(5:00~7:00, 계절 변동): 1일 20명 한정

### 문제 1-3 : FewShotPromptTemplate과 시스템 메시지 활용

In [ ]:
# FewShotChatMessagePromptTemplate 사용하는 경우
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI

examples = [
    {
        "news": "미국 연방준비제도(Fed)가 지속되는 인플레이션 압력에 대응하기 위해 기준금리를 0.25%p 추가 인상했다. \
            시장에서는 이번 결정이 글로벌 경제에 미칠 파장에 대해 예의주시하고 있다.",
        "keywords": "연방준비제도, 기준금리, 인플레이션"
    },
    {
        "news": "유럽연합(EU)이 2035년부터 내연기관 신차 판매를 전면 금지하는 법안을 최종 통과시켰다. \
            이는 기후 변화에 대응하고 탄소 중립 목표를 달성하기 위한 강력한 의지를 보여주는 조치로 평가된다.",
        "keywords": "유럽연합, 내연기관차, 탄소중립"
    },
    {
        "news": "국내 연구진이 개발한 새로운 알츠하이머 치료제가 미국 식품의약국(FDA)의 긴급사용승인을 받았다. \
            초기 단계 환자들의 인지 능력 저하를 늦추는 데 상당한 효과가 있는 것으로 나타나 기대를 모으고 있다.",
        "keywords": "알츠하이머, 치료제, 미국 식품의약국"
    }
]

# 예제 프롬프트 템플릿
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{news}"),
        ("ai", "키워드: {keywords}"),
    ]
)

# FewShotChatMessagePromptTemplate 적용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 구성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "뉴스 키워드 추출 전문가입니다. 핵심 키워드 3개를 추출하세요."),
        few_shot_prompt,
        ("human", "{news}"),
    ]
)

# 모델 생성 및 체인 구성
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
chain = final_prompt | llm

# 테스트 실행
result = chain.invoke({"news": "제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) \
                       및 버텍스 AI(Vertex AI)에서 제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다. \
                       모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, \
                       텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. \
                       내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다."})
print(result.content)

키워드: 제미나이 2.0 플래시, 개발자 공개, 멀티모달
